# HSV 遮罩應用

**HSV簡介**
HSV即色相、飽和度、明度（英語：Hue, Saturation, Value），HSV最常應用於影像中提取特定顏色的目標物，
我們可以透過改變HSV中的三個分量（H、S、V）提取我們需要的顏色。

**HSV參數說明**
- 色相（H）是色彩的基本屬性，就是平常所說的顏色名稱，如紅色、黃色等。
- 飽和度（S）是指色彩的純度，越高色彩越純，低則逐漸變灰，取0-100%的數值。
- 明度（V），亮度（L），取0-100%。

### 1. 載入模組

In [22]:
import numpy as np
import cv2 as cv


### 2. 載入影像並轉換成 HSV 格式

In [23]:
# 讀取影像
img = cv.imread('./img/balls.jpg')
img = cv.resize(img, (640, 480), interpolation=cv.INTER_AREA)

# 先將原影像由 BGR 色域轉成 HSV 色域並顯示
hsv_img = cv.cvtColor(img, cv.COLOR_BGR2HSV)


### 3. 定義物體顏色於 HSV 色域中的上下界

若要保留藍色安全帽的部份，首先需要先取得影像遮罩，以及定義色彩空間（藍色）的上下界。

In [33]:
# 定義藍色部份 HSV_mask 的上下限值
lower_blue = np.array([110,125,84])
upper_blue = np.array([122,255,200])

# 透過 inRange() 函式形成白色遮罩，遮罩區域為原始影像藍色區域
hsv_bin_mask = cv.inRange(hsv_img, lower_blue, upper_blue)

# 顯示影像
cv.imshow("HSV_mask",hsv_bin_mask)

# 按下任意鍵關閉視窗
cv.waitKey(0)
cv.destroyAllWindows()


### 4. 依據顏色形成遮罩

過濾之後將取得的遮罩與原圖做合併，即得到下圖過濾後的結果。

In [25]:
# 套用 hsv_bin_mask 到原圖上進行過濾
hsv_mask = cv.bitwise_and(img, img, None, mask=hsv_bin_mask)

# 顯示過濾後的影像
cv.imshow("HSV_mask_result", hsv_mask)

# 按下任意鍵關閉視窗
cv.waitKey(0)
cv.destroyAllWindows()


### 5. 框出遮罩區域

我們先前利用HSV顏色過濾這個特性將藍色的部份擷取出來後，接下來可以在原圖檔上框出來

In [38]:
# 將我們的 HSV Mask 套用OpenCV內建的Contours輪廓函式，在我們目標物體畫出邊框

# 找輪廓
(cnts, _) = cv.findContours(hsv_bin_mask.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)  # 回傳輪廓list

copy_img_1=img.copy()
copy_img_2 = img.copy()
clone= img.copy()

cv.drawContours(clone, cnts, -1, (0, 255, 0), 2)

for cnt in cnts:
    (x, y, w, h) = cv.boundingRect(cnt)  # 矩形 rect
    cv.rectangle(copy_img_1, (x, y), (x + w, y + h), (0, 0, 255), 1)  # 畫矩形
    
    BoundingBox = cv.minAreaRect(cnt)  # 最小外接矩形的中心（x，y），（寬度，高度），旋轉角度）
    BoundingBox = np.int0(cv.boxPoints(BoundingBox))  # int0 會省略小數點後方的數字
    cv.drawContours(copy_img_2, [BoundingBox], -1, (0, 255, 0), 1)

# 顯示影像
cv.imshow('Contour', clone)  # 輪廓
cv.imshow('Bounding Rectangle', copy_img_1)  # 矩形
cv.imshow('Minimum Area Rectangle', copy_img_2)  # 最小外接矩形

# 按下任意鍵關閉視窗
cv.waitKey(0)
cv.destroyAllWindows()
